In [1]:
import pandas as pd
import numpy as np
import os


import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt 
from scipy import stats
from sklearn.linear_model import LinearRegression

from statsmodels.formula.api import ols as sm_ols
from statsmodels.iolib.summary2 import summary_col # nicer tables
import statsmodels.api as sm

# importing required modules
from zipfile import ZipFile
import zipfile

In [11]:
# file_name = 'stataversion.dta'
# returns_data = pd.read_stata(file_name) #reads in the data file 
# returns_data.head()

,permno,date,siccd,ncusip,ticker,comnam,shrcls,naics,prc,vol,ret,shrout,cfacpr,cfacshr,sprtrn
0,10026,20180102,2052,46603210,JJSF,J & J SNACK FOODS CORP,,311821.0,149.17999,190618.0,-0.017454,18668.0,1.0,1.0,0.008303
1,10026,20180103,2052,46603210,JJSF,J & J SNACK FOODS CORP,,311821.0,147.69000,63693.0,-0.009988,18668.0,1.0,1.0,0.006399
2,10026,20180104,2052,46603210,JJSF,J & J SNACK FOODS CORP,,311821.0,149.73000,127552.0,0.013813,18668.0,1.0,1.0,0.004029
3,10026,20180105,2052,46603210,JJSF,J & J SNACK FOODS CORP,,311821.0,148.30000,44647.0,-0.009550,18668.0,1.0,1.0,0.007034
4,10026,20180108,2052,46603210,JJSF,J & J SNACK FOODS CORP,,311821.0,148.41000,55014.0,0.000742,18668.0,1.0,1.0,0.001662


In [ ]:
file_name2 = 'hpbx8j61yn2jkyar.dta'
fundamental_data = pd.read_stata(file_name2) #reads in the data file 
fundamental_data.head()

In [13]:

  
# specifying the zip file name
file_name = "fundamentals.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    print('Extracting all the files now...')
    zip.extractall()
    #data = zip.read('hpbx8j61yn2jkyar.dta')
    extracted_file = zipfile.open('hpbx8j61yn2jkyar.dta') #pull the dta file #
    
    

File Name                                             Modified             Size
hpbx8j61yn2jkyar.dta                           2021-04-06 14:56:06   4401995759
Extracting all the files now...


AttributeError: module 'zipfile' has no attribute 'open'

In [2]:
reader=pd.read_stata("hpbx8j61yn2jkyar.dta",chunksize=100000) #read dta file in chunks

df = pd.DataFrame()

for itm in reader:
    df=df.append(itm)

df.to_csv("fundamentals.csv") #convert it to a csv 

In [4]:
reader=pd.read_stata("stataversion.dta",chunksize=100000)

df = pd.DataFrame()

for itm in reader:
    df=df.append(itm)

df.to_csv("returns.csv")

In [7]:
fundamentals = pd.read_csv('fundamentals.csv', low_memory=False)
fundamentals.head()

,Unnamed: 0,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,...,prirow,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,ipodate
0,0,1004,2010-02-28,2009.0,3.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01
1,1,1004,2010-05-31,2009.0,4.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01
2,2,1004,2010-08-31,2010.0,1.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01
3,3,1004,2010-11-30,2010.0,2.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01
4,4,1004,2011-02-28,2010.0,3.0,5.0,INDL,C,D,STD,...,NaN,1.0,5080.0,110.0,925.0,B,IL,0.0,www.aarcorp.com,1988-01-01


In [11]:
returns = pd.read_csv('returns.csv', low_memory=False)
returns.head()

,Unnamed: 0,permno,date,siccd,ncusip,ticker,comnam,shrcls,naics,prc,vol,ret,shrout,cfacpr,cfacshr,sprtrn
0,0,10026,20180102,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,149.17999,190618.0,-0.017454,18668.0,1.0,1.0,0.008303
1,1,10026,20180103,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,147.69000,63693.0,-0.009988,18668.0,1.0,1.0,0.006399
2,2,10026,20180104,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,149.73000,127552.0,0.013813,18668.0,1.0,1.0,0.004029
3,3,10026,20180105,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,148.30000,44647.0,-0.009550,18668.0,1.0,1.0,0.007034
4,4,10026,20180108,2052,46603210,JJSF,J & J SNACK FOODS CORP,NaN,311821.0,148.41000,55014.0,0.000742,18668.0,1.0,1.0,0.001662


In [10]:
fundamentals.shape
returns.shape

NameError: name 'returns' is not defined